In [110]:
import quantiacsToolbox
import pandas as pd
import numpy as np
import os
from os.path import dirname, abspath
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
import pickle
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers import Dropout
import tensorflow
from keras.models import model_from_json
from keras.preprocessing.sequence import TimeseriesGenerator

In [80]:
import tensorflow

# Data Preparation

In [6]:
markets = ['CASH','F_AD','F_BO','F_BP','F_C','F_CC','F_CD','F_CL','F_CT','F_DX','F_EC',
                           'F_ED','F_ES','F_FC','F_FV','F_GC','F_HG','F_HO','F_JY','F_KC','F_LB','F_LC',
                           'F_LN','F_MD','F_MP','F_NG','F_NQ','F_NR','F_O','F_OJ','F_PA','F_PL','F_RB',
                           'F_RU','F_S','F_SB','F_SF','F_SI','F_SM','F_TU','F_TY','F_US','F_W','F_XX',
                           'F_YM','F_AX','F_CA','F_DT','F_UB','F_UZ','F_GS','F_LX','F_SS','F_DL','F_ZQ',
                           'F_VX','F_AE','F_BG','F_BC','F_LU','F_DM','F_AH','F_CF','F_DZ','F_FB','F_FL',
                           'F_FM','F_FP','F_FY','F_GX','F_HP','F_LR','F_LQ','F_ND','F_NY','F_PQ','F_RR',
                           'F_RF','F_RP','F_RY','F_SH','F_SX','F_TR','F_EB','F_VF','F_VT','F_VW','F_GD','F_F']

In [45]:
train_begin = '20180101'
train_end = '20201231'

In [46]:
data = quantiacsToolbox.loadData(markets, dataToLoad = ['DATE','OPEN','HIGH', 'LOW', 'CLOSE'], beginInSample = train_begin, endInSample = train_end)

Loading Data...
Done!


In [47]:
data['CLOSE'].shape

(2586, 89)

In [146]:
date = np.array(list(map(lambda x : str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:], data['DATE'])))

In [49]:
df = pd.DataFrame(data['CLOSE'], columns = markets, index = date)

In [ ]:
df.head()

In [51]:
df.to_csv('data_'+train_begin+'_'+train_end+'.csv')

In [57]:
train_begin = '20180101'
train_end = '20201231'
df = pd.read_csv('data_'+train_begin+'_'+train_end+'.csv')
df = df.rename(columns = {'Unnamed: 0':'date'})
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

# One-Step Forecast with LSTM

In [58]:
scaler = MinMaxScaler(feature_range=(0, 1))

In [59]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [60]:
lstm_models = {el:None for el in markets[1:]}

In [147]:
date[-102]

'2020-08-11'

In [155]:
for market in markets[1:]:
    print('training for: ' + market)
    dataset = df[[market]].loc[:'20200811',:].values.astype('float32')
    dataset = scaler.fit_transform(dataset)
    lookback = 1
    X, Y = create_dataset(dataset, lookback)
    X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
    model = Sequential()
    model.add(LSTM(20, input_shape=(1, lookback)))
    model.add(Dense(20))
    model.add(Dense(20))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, Y, epochs=50, batch_size=1, verbose=2)
    lstm_models[market] = model

training for: F_AD
Epoch 1/50
673/673 - 2s - loss: 0.0336
Epoch 2/50
673/673 - 1s - loss: 6.8663e-04
Epoch 3/50
673/673 - 1s - loss: 4.1400e-04
Epoch 4/50
673/673 - 1s - loss: 3.8559e-04
Epoch 5/50
673/673 - 1s - loss: 4.4275e-04
Epoch 6/50
673/673 - 1s - loss: 4.3039e-04
Epoch 7/50
673/673 - 1s - loss: 4.3740e-04
Epoch 8/50
673/673 - 1s - loss: 4.5424e-04
Epoch 9/50
673/673 - 1s - loss: 4.2933e-04
Epoch 10/50
673/673 - 1s - loss: 4.2051e-04
Epoch 11/50
673/673 - 1s - loss: 4.1772e-04
Epoch 12/50
673/673 - 1s - loss: 4.2823e-04
Epoch 13/50
673/673 - 1s - loss: 4.2830e-04
Epoch 14/50
673/673 - 1s - loss: 4.0152e-04
Epoch 15/50
673/673 - 1s - loss: 4.2061e-04
Epoch 16/50
673/673 - 1s - loss: 4.2943e-04
Epoch 17/50
673/673 - 1s - loss: 4.0271e-04
Epoch 18/50
673/673 - 1s - loss: 4.1766e-04
Epoch 19/50
673/673 - 1s - loss: 4.2863e-04
Epoch 20/50
673/673 - 1s - loss: 3.9705e-04
Epoch 21/50
673/673 - 1s - loss: 4.0895e-04
Epoch 22/50
673/673 - 1s - loss: 4.6554e-04
Epoch 23/50
673/673 - 1s -

673/673 - 1s - loss: 0.0012
Epoch 46/50
673/673 - 1s - loss: 0.0012
Epoch 47/50
673/673 - 1s - loss: 0.0012
Epoch 48/50
673/673 - 1s - loss: 0.0012
Epoch 49/50
673/673 - 1s - loss: 0.0012
Epoch 50/50
673/673 - 1s - loss: 0.0012
training for: F_CC
Epoch 1/50
673/673 - 2s - loss: 0.0285
Epoch 2/50
673/673 - 1s - loss: 0.0020
Epoch 3/50
673/673 - 1s - loss: 0.0020
Epoch 4/50
673/673 - 1s - loss: 0.0021
Epoch 5/50
673/673 - 1s - loss: 0.0020
Epoch 6/50
673/673 - 1s - loss: 0.0020
Epoch 7/50
673/673 - 1s - loss: 0.0020
Epoch 8/50
673/673 - 1s - loss: 0.0019
Epoch 9/50
673/673 - 1s - loss: 0.0021
Epoch 10/50
673/673 - 1s - loss: 0.0020
Epoch 11/50
673/673 - 1s - loss: 0.0020
Epoch 12/50
673/673 - 1s - loss: 0.0020
Epoch 13/50
673/673 - 1s - loss: 0.0020
Epoch 14/50
673/673 - 1s - loss: 0.0020
Epoch 15/50
673/673 - 1s - loss: 0.0021
Epoch 16/50
673/673 - 1s - loss: 0.0020
Epoch 17/50
673/673 - 1s - loss: 0.0019
Epoch 18/50
673/673 - 1s - loss: 0.0019
Epoch 19/50
673/673 - 1s - loss: 0.0019
Ep

673/673 - 1s - loss: 6.3749e-04
Epoch 37/50
673/673 - 1s - loss: 5.9304e-04
Epoch 38/50
673/673 - 1s - loss: 6.4993e-04
Epoch 39/50
673/673 - 1s - loss: 6.4707e-04
Epoch 40/50
673/673 - 1s - loss: 5.9606e-04
Epoch 41/50
673/673 - 1s - loss: 6.6051e-04
Epoch 42/50
673/673 - 1s - loss: 5.9910e-04
Epoch 43/50
673/673 - 1s - loss: 5.9553e-04
Epoch 44/50
673/673 - 1s - loss: 6.2591e-04
Epoch 45/50
673/673 - 1s - loss: 6.7953e-04
Epoch 46/50
673/673 - 1s - loss: 6.0735e-04
Epoch 47/50
673/673 - 1s - loss: 6.1955e-04
Epoch 48/50
673/673 - 1s - loss: 6.6620e-04
Epoch 49/50
673/673 - 1s - loss: 5.9748e-04
Epoch 50/50
673/673 - 1s - loss: 5.8067e-04
training for: F_DX
Epoch 1/50
673/673 - 2s - loss: 0.0242
Epoch 2/50
673/673 - 1s - loss: 7.8426e-04
Epoch 3/50
673/673 - 1s - loss: 7.6601e-04
Epoch 4/50
673/673 - 1s - loss: 7.2073e-04
Epoch 5/50
673/673 - 1s - loss: 8.1028e-04
Epoch 6/50
673/673 - 1s - loss: 7.9274e-04
Epoch 7/50
673/673 - 1s - loss: 7.6298e-04
Epoch 8/50
673/673 - 1s - loss: 7.61

Epoch 24/50
673/673 - 1s - loss: 0.0014
Epoch 25/50
673/673 - 1s - loss: 0.0014
Epoch 26/50
673/673 - 1s - loss: 0.0014
Epoch 27/50
673/673 - 1s - loss: 0.0013
Epoch 28/50
673/673 - 1s - loss: 0.0013
Epoch 29/50
673/673 - 1s - loss: 0.0014
Epoch 30/50
673/673 - 1s - loss: 0.0014
Epoch 31/50
673/673 - 1s - loss: 0.0013
Epoch 32/50
673/673 - 1s - loss: 0.0013
Epoch 33/50
673/673 - 1s - loss: 0.0014
Epoch 34/50
673/673 - 1s - loss: 0.0013
Epoch 35/50
673/673 - 1s - loss: 0.0014
Epoch 36/50
673/673 - 1s - loss: 0.0013
Epoch 37/50
673/673 - 1s - loss: 0.0014
Epoch 38/50
673/673 - 1s - loss: 0.0013
Epoch 39/50
673/673 - 1s - loss: 0.0013
Epoch 40/50
673/673 - 1s - loss: 0.0013
Epoch 41/50
673/673 - 1s - loss: 0.0014
Epoch 42/50
673/673 - 1s - loss: 0.0014
Epoch 43/50
673/673 - 1s - loss: 0.0013
Epoch 44/50
673/673 - 1s - loss: 0.0013
Epoch 45/50
673/673 - 1s - loss: 0.0013
Epoch 46/50
673/673 - 1s - loss: 0.0014
Epoch 47/50
673/673 - 1s - loss: 0.0013
Epoch 48/50
673/673 - 1s - loss: 0.0013


Epoch 17/50
673/673 - 1s - loss: 8.9675e-04
Epoch 18/50
673/673 - 1s - loss: 9.2698e-04
Epoch 19/50
673/673 - 1s - loss: 9.6848e-04
Epoch 20/50
673/673 - 1s - loss: 8.9460e-04
Epoch 21/50
673/673 - 1s - loss: 8.8116e-04
Epoch 22/50
673/673 - 1s - loss: 9.4193e-04
Epoch 23/50
673/673 - 1s - loss: 8.9831e-04
Epoch 24/50
673/673 - 1s - loss: 8.9198e-04
Epoch 25/50
673/673 - 1s - loss: 9.6777e-04
Epoch 26/50
673/673 - 1s - loss: 9.2309e-04
Epoch 27/50
673/673 - 1s - loss: 9.0284e-04
Epoch 28/50
673/673 - 1s - loss: 9.1023e-04
Epoch 29/50
673/673 - 1s - loss: 9.3273e-04
Epoch 30/50
673/673 - 1s - loss: 9.1274e-04
Epoch 31/50
673/673 - 1s - loss: 9.2406e-04
Epoch 32/50
673/673 - 1s - loss: 9.2830e-04
Epoch 33/50
673/673 - 1s - loss: 9.7698e-04
Epoch 34/50
673/673 - 1s - loss: 9.1663e-04
Epoch 35/50
673/673 - 1s - loss: 8.9234e-04
Epoch 36/50
673/673 - 1s - loss: 8.9779e-04
Epoch 37/50
673/673 - 1s - loss: 9.4050e-04
Epoch 38/50
673/673 - 1s - loss: 8.6858e-04
Epoch 39/50
673/673 - 1s - loss:

...

Epoch 23/50
673/673 - 1s - loss: 8.9073e-04
Epoch 24/50
673/673 - 1s - loss: 8.6708e-04
Epoch 25/50
673/673 - 1s - loss: 8.9504e-04
Epoch 26/50
673/673 - 1s - loss: 8.4512e-04
Epoch 27/50
673/673 - 1s - loss: 8.8357e-04
Epoch 28/50
673/673 - 1s - loss: 8.1994e-04
Epoch 29/50
673/673 - 1s - loss: 8.7854e-04
Epoch 30/50
673/673 - 1s - loss: 9.3196e-04
Epoch 31/50
673/673 - 1s - loss: 8.1352e-04
Epoch 32/50
673/673 - 1s - loss: 8.1963e-04
Epoch 33/50
673/673 - 1s - loss: 8.4947e-04
Epoch 34/50
673/673 - 1s - loss: 8.2567e-04
Epoch 35/50
673/673 - 1s - loss: 8.0283e-04
Epoch 36/50
673/673 - 1s - loss: 8.5770e-04
Epoch 37/50
673/673 - 1s - loss: 8.6289e-04
Epoch 38/50
673/673 - 1s - loss: 8.9265e-04
Epoch 39/50
673/673 - 1s - loss: 8.5374e-04
Epoch 40/50
673/673 - 1s - loss: 9.0778e-04
Epoch 41/50
673/673 - 1s - loss: 8.2750e-04
Epoch 42/50
673/673 - 1s - loss: 8.7648e-04
Epoch 43/50
673/673 - 1s - loss: 8.9039e-04
Epoch 44/50
673/673 - 1s - loss: 8.2257e-04
Epoch 45/50
673/673 - 1s - loss:

Epoch 10/50
673/673 - 1s - loss: 0.0017
Epoch 11/50
673/673 - 1s - loss: 0.0017
Epoch 12/50
673/673 - 1s - loss: 0.0017
Epoch 13/50
673/673 - 1s - loss: 0.0017
Epoch 14/50
673/673 - 1s - loss: 0.0017
Epoch 15/50
673/673 - 1s - loss: 0.0016
Epoch 16/50
673/673 - 1s - loss: 0.0017
Epoch 17/50
673/673 - 1s - loss: 0.0017
Epoch 18/50
673/673 - 1s - loss: 0.0017
Epoch 19/50
673/673 - 1s - loss: 0.0017
Epoch 20/50
673/673 - 1s - loss: 0.0017
Epoch 21/50
673/673 - 1s - loss: 0.0017
Epoch 22/50
673/673 - 1s - loss: 0.0017
Epoch 23/50
673/673 - 1s - loss: 0.0018
Epoch 24/50
673/673 - 1s - loss: 0.0017
Epoch 25/50
673/673 - 1s - loss: 0.0017
Epoch 26/50
673/673 - 1s - loss: 0.0017
Epoch 27/50
673/673 - 1s - loss: 0.0017
Epoch 28/50
673/673 - 1s - loss: 0.0017
Epoch 29/50
673/673 - 1s - loss: 0.0017
Epoch 30/50
673/673 - 1s - loss: 0.0017
Epoch 31/50
673/673 - 1s - loss: 0.0017
Epoch 32/50
673/673 - 1s - loss: 0.0017
Epoch 33/50
673/673 - 1s - loss: 0.0016
Epoch 34/50
673/673 - 1s - loss: 0.0017


Epoch 50/50
673/673 - 1s - loss: 5.4243e-04
training for: F_RP
Epoch 1/50
673/673 - 2s - loss: 0.0246
Epoch 2/50
673/673 - 1s - loss: 0.0019
Epoch 3/50
673/673 - 1s - loss: 0.0017
Epoch 4/50
673/673 - 1s - loss: 0.0018
Epoch 5/50
673/673 - 1s - loss: 0.0018
Epoch 6/50
673/673 - 1s - loss: 0.0017
Epoch 7/50
673/673 - 1s - loss: 0.0018
Epoch 8/50
673/673 - 1s - loss: 0.0018
Epoch 9/50
673/673 - 1s - loss: 0.0017
Epoch 10/50
673/673 - 1s - loss: 0.0017
Epoch 11/50
673/673 - 1s - loss: 0.0017
Epoch 12/50
673/673 - 1s - loss: 0.0018
Epoch 13/50
673/673 - 1s - loss: 0.0018
Epoch 14/50
673/673 - 1s - loss: 0.0017
Epoch 15/50
673/673 - 1s - loss: 0.0017
Epoch 16/50
673/673 - 1s - loss: 0.0017
Epoch 17/50
673/673 - 1s - loss: 0.0017
Epoch 18/50
673/673 - 1s - loss: 0.0017
Epoch 19/50
673/673 - 1s - loss: 0.0017
Epoch 20/50
673/673 - 1s - loss: 0.0017
Epoch 21/50
673/673 - 1s - loss: 0.0017
Epoch 22/50
673/673 - 1s - loss: 0.0017
Epoch 23/50
673/673 - 1s - loss: 0.0017
Epoch 24/50
673/673 - 1s -

Epoch 49/50
673/673 - 1s - loss: 0.0015
Epoch 50/50
673/673 - 1s - loss: 0.0015
training for: F_TR
Epoch 1/50
673/673 - 2s - loss: 0.0352
Epoch 2/50
673/673 - 1s - loss: 6.5625e-04
Epoch 3/50
673/673 - 1s - loss: 5.7577e-04
Epoch 4/50
673/673 - 1s - loss: 5.2233e-04
Epoch 5/50
673/673 - 1s - loss: 5.4910e-04
Epoch 6/50
673/673 - 1s - loss: 5.4955e-04
Epoch 7/50
673/673 - 1s - loss: 5.8191e-04
Epoch 8/50
673/673 - 1s - loss: 5.8431e-04
Epoch 9/50
673/673 - 1s - loss: 5.5827e-04
Epoch 10/50
673/673 - 1s - loss: 5.2983e-04
Epoch 11/50
673/673 - 1s - loss: 5.5390e-04
Epoch 12/50
673/673 - 1s - loss: 5.3238e-04
Epoch 13/50
673/673 - 1s - loss: 5.9790e-04
Epoch 14/50
673/673 - 1s - loss: 5.1764e-04
Epoch 15/50
673/673 - 1s - loss: 5.5331e-04
Epoch 16/50
673/673 - 1s - loss: 5.3211e-04
Epoch 17/50
673/673 - 1s - loss: 5.8311e-04
Epoch 18/50
673/673 - 1s - loss: 5.8267e-04
Epoch 19/50
673/673 - 1s - loss: 5.8349e-04
Epoch 20/50
673/673 - 1s - loss: 5.4957e-04
Epoch 21/50
673/673 - 1s - loss: 5

Epoch 40/50
673/673 - 1s - loss: 4.5065e-04
Epoch 41/50
673/673 - 1s - loss: 4.7420e-04
Epoch 42/50
673/673 - 1s - loss: 4.4179e-04
Epoch 43/50
673/673 - 1s - loss: 4.5137e-04
Epoch 44/50
673/673 - 1s - loss: 4.5427e-04
Epoch 45/50
673/673 - 1s - loss: 4.4397e-04
Epoch 46/50
673/673 - 1s - loss: 4.8399e-04
Epoch 47/50
673/673 - 1s - loss: 4.5495e-04
Epoch 48/50
673/673 - 1s - loss: 4.7627e-04
Epoch 49/50
673/673 - 1s - loss: 4.5716e-04
Epoch 50/50
673/673 - 1s - loss: 4.7451e-04
training for: F_VW
Epoch 1/50
673/673 - 4s - loss: 0.0272
Epoch 2/50
673/673 - 1s - loss: 6.3799e-04
Epoch 3/50
673/673 - 1s - loss: 6.1936e-04
Epoch 4/50
673/673 - 1s - loss: 6.5909e-04
Epoch 5/50
673/673 - 1s - loss: 6.8106e-04
Epoch 6/50
673/673 - 1s - loss: 6.9153e-04
Epoch 7/50
673/673 - 1s - loss: 6.7379e-04
Epoch 8/50
673/673 - 1s - loss: 6.3374e-04
Epoch 9/50
673/673 - 1s - loss: 6.5481e-04
Epoch 10/50
673/673 - 1s - loss: 6.2529e-04
Epoch 11/50
673/673 - 1s - loss: 6.6960e-04
Epoch 12/50
673/673 - 1s -

In [67]:
lstm_models

{'F_AD': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5f101749a0>,
 'F_BO': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5f103be910>,
 'F_BP': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5ed86011c0>,
 'F_C': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5f106086d0>,
 'F_CC': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5f1077bee0>,
 'F_CD': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5f10588ee0>,
 'F_CL': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5e7671f610>,
 'F_CT': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5e75e2d850>,
 'F_DX': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5e752a0f40>,
 'F_EC': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5e940a4100>,
 'F_ED': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5e7467bcd0>,
 'F_ES': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f5e7426afa0>,
 'F_F

In [156]:
for market in markets[1:]:
    model_json = lstm_models[market].to_json()
    with open('lstm_models/lstm_model_'+market+'.json', "w") as json_file:
        json_file.write(model_json)
    lstm_models[market].save_weights('lstm_models/lstm_model_'+market+'.h5')

In [235]:
# testScores = []
correct_dict = {el:None for el in markets[1:]}

In [250]:
for market in markets[1:]:
    correct = 0
    print(market)
    for i in range(len(df)-100, len(df)-1):
        dataset_test = df[[market]].loc[:df.index[i],:].values.astype('float32')
        dataset_test = scaler.fit_transform(dataset_test)
        lookback = 1
        X_test, Y_test = create_dataset(dataset_test, lookback)
        X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
        json_file = open('lstm_models/lstm_model_'+market+'.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        model.load_weights('lstm_models/lstm_model_'+market+'.h5')
        pred = model.predict(X_test)
        pred = scaler.inverse_transform(pred)
        pos = 1 if pred[-1,0] > df[market][i] else -1
        actual = 1 if df[market][i+1] > df[market][i] else -1
        if pos == actual:
            correct += 1
    correct_dict[market] = correct
#         testScore = math.sqrt(mean_squared_error(Y_test[0], pred[:,0]))
#         testScores.append(testScore)

F_AD
F_BO
F_BP
F_C
F_CC
F_CD
F_CL
F_CT
F_DX
F_EC
F_ED
F_ES
F_FC
F_FV
F_GC
F_HG
F_HO
F_JY
F_KC
F_LB
F_LC
F_LN
F_MD
F_MP
F_NG
F_NQ
F_NR
F_O
F_OJ
F_PA
F_PL
F_RB
F_RU
F_S
F_SB
F_SF
F_SI
F_SM
F_TU
F_TY
F_US
F_W
F_XX
F_YM
F_AX
F_CA
F_DT
F_UB
F_UZ
F_GS
F_LX
F_SS
F_DL
F_ZQ
F_VX
F_AE
F_BG
F_BC
F_LU
F_DM
F_AH
F_CF
F_DZ
F_FB
F_FL
F_FM
F_FP
F_FY
F_GX
F_HP
F_LR
F_LQ
F_ND
F_NY
F_PQ
F_RR
F_RF
F_RP
F_RY
F_SH
F_SX
F_TR
F_EB
F_VF
F_VT
F_VW
F_GD
F_F


In [252]:
correct_list = [0] + list(correct_dict.values())
correct_list = [max(0, x-50) for x in correct_list]
print(correct_list)

[0, 0, 1, 4, 0, 4, 8, 0, 0, 0, 0, 14, 0, 8, 0, 7, 0, 1, 0, 3, 0, 2, 11, 4, 0, 0, 1, 12, 0, 3, 10, 0, 4, 0, 0, 3, 0, 3, 0, 0, 2, 4, 5, 0, 0, 3, 4, 0, 1, 1, 0, 0, 1, 0, 28, 0, 1, 5, 3, 0, 0, 8, 0, 0, 0, 0, 6, 9, 5, 0, 0, 0, 11, 0, 2, 0, 0, 0, 5, 3, 5, 1, 8, 7, 3, 0, 0, 0, 0]


In [253]:
with open('weights_list_lstm.txt', 'w') as f:
    for correct in correct_list:
        f.write(str(correct) + '\n')

# Multi-Step Forecast with LSTM

In [211]:
def prepare_columns(data):
    df = pd.DataFrame(data)
    columns, names = list(), list()
    # input sequence
    for i in range(5, 0, -1):
        columns.append(df.shift(i))
        names += [('var(t-%d)' % (i))]
    # forecast sequence
    columns.append(df)
    names += [('var(t)')]
    con = pd.concat(columns, axis=1)
    con.columns = names
    con.dropna(inplace=True)
    return con

In [248]:
def prepare_data(scaler, data):
    values = data.values
    values = values.reshape(len(values), 1)
    scaled = scaler.fit_transform(values)
    scaled = scaled.reshape(len(scaled), 1)
    final = prepare_columns(scaled)
    final = final.values
    train, test = final[0:-100], final[-100:]
    return train, test

In [220]:
def fit_lstm(train):
    X, Y = train[:, 0:5], train[:, 5:]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(20, batch_input_shape=(1, X.shape[1], X.shape[2])))
    model.add(Dense(Y.shape[1]))
    model.add(Dense(Y.shape[1]))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, Y, epochs=50, batch_size=1, verbose=2, shuffle=False)
    return model

In [225]:
def forecast_lstm(model, train, test):
    forecasts = list()
    for i in range(len(test)):
        X, Y = test[i, 0:5], test[i, 5:]
        X = X.reshape(1, 1, len(X))
        pred = model.predict(X, batch_size=1)
        fore = [x for x in pred[0, :]]
        forecasts.append(fore)
    return forecasts

In [228]:
def inverse(series, forecasts, scaler):
    inverted = list()
    for i in range(len(forecasts)):
        forecast = np.array(forecasts[i])
        forecast = forecast.reshape(1, len(forecast))
        inv_scale = scaler.inverse_transform(forecast)
        inv_scale = inv_scale[0, :]
        inverted.append(inv_scale)
    return inverted

In [230]:
def calculate_rmse(test, forecasts):
    actual = [row[0] for row in test]
    pred = [fore[0] for fore in forecasts]
    rmse = math.sqrt(mean_squared_error(actual, pred))
    return rmse

In [222]:
rmse_list = []
multi_step_lstm_models = {el:None for el in markets[1:]}

In [232]:
for market in markets[1:]:
    print('fit for: ' + market)
    train, test = prepare_data(scaler, df[[market]])
    model = fit_lstm(train)
    forecasts = forecast_lstm(model, train, test)
    forecasts = inverse(df[[market]], forecasts, scaler)
    actual = [row[5:] for row in test]
    actual = inverse(df[[market]], actual, scaler)
    rmse = calculate_rmse(actual, forecasts)
    multi_step_lstm_models[market] = model
    rmse_list.append(rmse)

fit for: F_AD
Epoch 1/50
671/671 - 2s - loss: 0.0678
Epoch 2/50
671/671 - 1s - loss: 0.0088
Epoch 3/50
671/671 - 1s - loss: 0.0047
Epoch 4/50
671/671 - 1s - loss: 0.0046
Epoch 5/50
671/671 - 1s - loss: 0.0045
Epoch 6/50
671/671 - 1s - loss: 0.0043
Epoch 7/50
671/671 - 1s - loss: 0.0040
Epoch 8/50
671/671 - 1s - loss: 0.0038
Epoch 9/50
671/671 - 1s - loss: 0.0035
Epoch 10/50
671/671 - 1s - loss: 0.0032
Epoch 11/50
671/671 - 1s - loss: 0.0030
Epoch 12/50
671/671 - 1s - loss: 0.0028
Epoch 13/50
671/671 - 1s - loss: 0.0027
Epoch 14/50
671/671 - 1s - loss: 0.0025
Epoch 15/50
671/671 - 1s - loss: 0.0024
Epoch 16/50
671/671 - 1s - loss: 0.0024
Epoch 17/50
671/671 - 1s - loss: 0.0023
Epoch 18/50
671/671 - 1s - loss: 0.0023
Epoch 19/50
671/671 - 1s - loss: 0.0023
Epoch 20/50
671/671 - 1s - loss: 0.0023
Epoch 21/50
671/671 - 1s - loss: 0.0023
Epoch 22/50
671/671 - 1s - loss: 0.0023
Epoch 23/50
671/671 - 1s - loss: 0.0023
Epoch 24/50
671/671 - 1s - loss: 0.0023
Epoch 25/50
671/671 - 1s - loss: 0.

671/671 - 1s - loss: 0.0079
Epoch 6/50
671/671 - 1s - loss: 0.0078
Epoch 7/50
671/671 - 1s - loss: 0.0079
Epoch 8/50
671/671 - 1s - loss: 0.0083
Epoch 9/50
671/671 - 1s - loss: 0.0090
Epoch 10/50
671/671 - 1s - loss: 0.0099
Epoch 11/50
671/671 - 1s - loss: 0.0107
Epoch 12/50
671/671 - 1s - loss: 0.0113
Epoch 13/50
671/671 - 1s - loss: 0.0117
Epoch 14/50
671/671 - 1s - loss: 0.0120
Epoch 15/50
671/671 - 1s - loss: 0.0123
Epoch 16/50
671/671 - 1s - loss: 0.0126
Epoch 17/50
671/671 - 1s - loss: 0.0128
Epoch 18/50
671/671 - 1s - loss: 0.0131
Epoch 19/50
671/671 - 1s - loss: 0.0133
Epoch 20/50
671/671 - 1s - loss: 0.0135
Epoch 21/50
671/671 - 1s - loss: 0.0136
Epoch 22/50
671/671 - 1s - loss: 0.0138
Epoch 23/50
671/671 - 1s - loss: 0.0139
Epoch 24/50
671/671 - 1s - loss: 0.0140
Epoch 25/50
671/671 - 1s - loss: 0.0141
Epoch 26/50
671/671 - 1s - loss: 0.0142
Epoch 27/50
671/671 - 1s - loss: 0.0143
Epoch 28/50
671/671 - 1s - loss: 0.0144
Epoch 29/50
671/671 - 1s - loss: 0.0144
Epoch 30/50
671/

Epoch 10/50
671/671 - 1s - loss: 0.0043
Epoch 11/50
671/671 - 1s - loss: 0.0044
Epoch 12/50
671/671 - 1s - loss: 0.0045
Epoch 13/50
671/671 - 1s - loss: 0.0046
Epoch 14/50
671/671 - 1s - loss: 0.0047
Epoch 15/50
671/671 - 1s - loss: 0.0048
Epoch 16/50
671/671 - 1s - loss: 0.0049
Epoch 17/50
671/671 - 1s - loss: 0.0050
Epoch 18/50
671/671 - 1s - loss: 0.0051
Epoch 19/50
671/671 - 1s - loss: 0.0052
Epoch 20/50
671/671 - 1s - loss: 0.0053
Epoch 21/50
671/671 - 1s - loss: 0.0054
Epoch 22/50
671/671 - 1s - loss: 0.0054
Epoch 23/50
671/671 - 1s - loss: 0.0055
Epoch 24/50
671/671 - 1s - loss: 0.0056
Epoch 25/50
671/671 - 1s - loss: 0.0057
Epoch 26/50
671/671 - 1s - loss: 0.0057
Epoch 27/50
671/671 - 1s - loss: 0.0058
Epoch 28/50
671/671 - 1s - loss: 0.0058
Epoch 29/50
671/671 - 1s - loss: 0.0058
Epoch 30/50
671/671 - 1s - loss: 0.0059
Epoch 31/50
671/671 - 1s - loss: 0.0059
Epoch 32/50
671/671 - 1s - loss: 0.0059
Epoch 33/50
671/671 - 1s - loss: 0.0059
Epoch 34/50
671/671 - 1s - loss: 0.0059


671/671 - 1s - loss: 0.0076
Epoch 15/50
671/671 - 1s - loss: 0.0076
Epoch 16/50
671/671 - 1s - loss: 0.0076
Epoch 17/50
671/671 - 1s - loss: 0.0076
Epoch 18/50
671/671 - 1s - loss: 0.0076
Epoch 19/50
671/671 - 1s - loss: 0.0076
Epoch 20/50
671/671 - 1s - loss: 0.0076
Epoch 21/50
671/671 - 1s - loss: 0.0076
Epoch 22/50
671/671 - 1s - loss: 0.0076
Epoch 23/50
671/671 - 1s - loss: 0.0076
Epoch 24/50
671/671 - 1s - loss: 0.0076
Epoch 25/50
671/671 - 1s - loss: 0.0076
Epoch 26/50
671/671 - 1s - loss: 0.0075
Epoch 27/50
671/671 - 1s - loss: 0.0075
Epoch 28/50
671/671 - 1s - loss: 0.0075
Epoch 29/50
671/671 - 1s - loss: 0.0075
Epoch 30/50
671/671 - 1s - loss: 0.0075
Epoch 31/50
671/671 - 1s - loss: 0.0075
Epoch 32/50
671/671 - 1s - loss: 0.0075
Epoch 33/50
671/671 - 1s - loss: 0.0075
Epoch 34/50
671/671 - 1s - loss: 0.0075
Epoch 35/50
671/671 - 1s - loss: 0.0075
Epoch 36/50
671/671 - 1s - loss: 0.0075
Epoch 37/50
671/671 - 1s - loss: 0.0074
Epoch 38/50
671/671 - 1s - loss: 0.0074
Epoch 39/50


Epoch 19/50
671/671 - 1s - loss: 0.0392
Epoch 20/50
671/671 - 1s - loss: 0.0394
Epoch 21/50
671/671 - 1s - loss: 0.0395
Epoch 22/50
671/671 - 1s - loss: 0.0396
Epoch 23/50
671/671 - 1s - loss: 0.0396
Epoch 24/50
671/671 - 1s - loss: 0.0396
Epoch 25/50
671/671 - 1s - loss: 0.0395
Epoch 26/50
671/671 - 1s - loss: 0.0395
Epoch 27/50
671/671 - 1s - loss: 0.0394
Epoch 28/50
671/671 - 1s - loss: 0.0393
Epoch 29/50
671/671 - 1s - loss: 0.0393
Epoch 30/50
671/671 - 1s - loss: 0.0394
Epoch 31/50
671/671 - 1s - loss: 0.0394
Epoch 32/50
671/671 - 1s - loss: 0.0394
Epoch 33/50
671/671 - 1s - loss: 0.0394
Epoch 34/50
671/671 - 1s - loss: 0.0392
Epoch 35/50
671/671 - 1s - loss: 0.0390
Epoch 36/50
671/671 - 1s - loss: 0.0386
Epoch 37/50
671/671 - 1s - loss: 0.0383
Epoch 38/50
671/671 - 1s - loss: 0.0379
Epoch 39/50
671/671 - 1s - loss: 0.0375
Epoch 40/50
671/671 - 1s - loss: 0.0371
Epoch 41/50
671/671 - 1s - loss: 0.0367
Epoch 42/50
671/671 - 1s - loss: 0.0364
Epoch 43/50
671/671 - 1s - loss: 0.0360


...

671/671 - 1s - loss: 0.0352
Epoch 16/50
671/671 - 1s - loss: 0.0356
Epoch 17/50
671/671 - 1s - loss: 0.0358
Epoch 18/50
671/671 - 1s - loss: 0.0359
Epoch 19/50
671/671 - 1s - loss: 0.0358
Epoch 20/50
671/671 - 1s - loss: 0.0356
Epoch 21/50
671/671 - 1s - loss: 0.0353
Epoch 22/50
671/671 - 1s - loss: 0.0349
Epoch 23/50
671/671 - 1s - loss: 0.0346
Epoch 24/50
671/671 - 1s - loss: 0.0342
Epoch 25/50
671/671 - 1s - loss: 0.0338
Epoch 26/50
671/671 - 1s - loss: 0.0334
Epoch 27/50
671/671 - 1s - loss: 0.0330
Epoch 28/50
671/671 - 1s - loss: 0.0326
Epoch 29/50
671/671 - 1s - loss: 0.0323
Epoch 30/50
671/671 - 1s - loss: 0.0319
Epoch 31/50
671/671 - 1s - loss: 0.0315
Epoch 32/50
671/671 - 1s - loss: 0.0311
Epoch 33/50
671/671 - 1s - loss: 0.0307
Epoch 34/50
671/671 - 1s - loss: 0.0303
Epoch 35/50
671/671 - 1s - loss: 0.0300
Epoch 36/50
671/671 - 1s - loss: 0.0296
Epoch 37/50
671/671 - 1s - loss: 0.0292
Epoch 38/50
671/671 - 1s - loss: 0.0288
Epoch 39/50
671/671 - 1s - loss: 0.0285
Epoch 40/50


Epoch 20/50
671/671 - 1s - loss: 0.0450
Epoch 21/50
671/671 - 1s - loss: 0.0445
Epoch 22/50
671/671 - 1s - loss: 0.0440
Epoch 23/50
671/671 - 1s - loss: 0.0436
Epoch 24/50
671/671 - 1s - loss: 0.0432
Epoch 25/50
671/671 - 1s - loss: 0.0429
Epoch 26/50
671/671 - 1s - loss: 0.0425
Epoch 27/50
671/671 - 1s - loss: 0.0422
Epoch 28/50
671/671 - 1s - loss: 0.0419
Epoch 29/50
671/671 - 1s - loss: 0.0416
Epoch 30/50
671/671 - 1s - loss: 0.0413
Epoch 31/50
671/671 - 1s - loss: 0.0411
Epoch 32/50
671/671 - 1s - loss: 0.0408
Epoch 33/50
671/671 - 1s - loss: 0.0406
Epoch 34/50
671/671 - 1s - loss: 0.0403
Epoch 35/50
671/671 - 1s - loss: 0.0401
Epoch 36/50
671/671 - 1s - loss: 0.0399
Epoch 37/50
671/671 - 1s - loss: 0.0397
Epoch 38/50
671/671 - 1s - loss: 0.0395
Epoch 39/50
671/671 - 1s - loss: 0.0393
Epoch 40/50
671/671 - 1s - loss: 0.0391
Epoch 41/50
671/671 - 1s - loss: 0.0390
Epoch 42/50
671/671 - 1s - loss: 0.0388
Epoch 43/50
671/671 - 1s - loss: 0.0386
Epoch 44/50
671/671 - 1s - loss: 0.0385


Epoch 20/50
671/671 - 1s - loss: 0.0164
Epoch 21/50
671/671 - 1s - loss: 0.0166
Epoch 22/50
671/671 - 1s - loss: 0.0168
Epoch 23/50
671/671 - 1s - loss: 0.0170
Epoch 24/50
671/671 - 1s - loss: 0.0171
Epoch 25/50
671/671 - 1s - loss: 0.0173
Epoch 26/50
671/671 - 1s - loss: 0.0174
Epoch 27/50
671/671 - 1s - loss: 0.0175
Epoch 28/50
671/671 - 1s - loss: 0.0177
Epoch 29/50
671/671 - 1s - loss: 0.0179
Epoch 30/50
671/671 - 1s - loss: 0.0181
Epoch 31/50
671/671 - 1s - loss: 0.0182
Epoch 32/50
671/671 - 1s - loss: 0.0182
Epoch 33/50
671/671 - 1s - loss: 0.0183
Epoch 34/50
671/671 - 1s - loss: 0.0183
Epoch 35/50
671/671 - 1s - loss: 0.0183
Epoch 36/50
671/671 - 1s - loss: 0.0184
Epoch 37/50
671/671 - 1s - loss: 0.0184
Epoch 38/50
671/671 - 1s - loss: 0.0184
Epoch 39/50
671/671 - 1s - loss: 0.0184
Epoch 40/50
671/671 - 1s - loss: 0.0184
Epoch 41/50
671/671 - 1s - loss: 0.0184
Epoch 42/50
671/671 - 1s - loss: 0.0184
Epoch 43/50
671/671 - 1s - loss: 0.0184
Epoch 44/50
671/671 - 1s - loss: 0.0184


Epoch 20/50
671/671 - 1s - loss: 0.0116
Epoch 21/50
671/671 - 1s - loss: 0.0120
Epoch 22/50
671/671 - 1s - loss: 0.0124
Epoch 23/50
671/671 - 1s - loss: 0.0130
Epoch 24/50
671/671 - 1s - loss: 0.0137
Epoch 25/50
671/671 - 1s - loss: 0.0143
Epoch 26/50
671/671 - 1s - loss: 0.0147
Epoch 27/50
671/671 - 1s - loss: 0.0151
Epoch 28/50
671/671 - 1s - loss: 0.0152
Epoch 29/50
671/671 - 1s - loss: 0.0153
Epoch 30/50
671/671 - 1s - loss: 0.0153
Epoch 31/50
671/671 - 1s - loss: 0.0153
Epoch 32/50
671/671 - 1s - loss: 0.0153
Epoch 33/50
671/671 - 1s - loss: 0.0153
Epoch 34/50
671/671 - 1s - loss: 0.0153
Epoch 35/50
671/671 - 1s - loss: 0.0153
Epoch 36/50
671/671 - 1s - loss: 0.0153
Epoch 37/50
671/671 - 1s - loss: 0.0154
Epoch 38/50
671/671 - 1s - loss: 0.0156
Epoch 39/50
671/671 - 1s - loss: 0.0158
Epoch 40/50
671/671 - 1s - loss: 0.0158
Epoch 41/50
671/671 - 1s - loss: 0.0156
Epoch 42/50
671/671 - 1s - loss: 0.0155
Epoch 43/50
671/671 - 1s - loss: 0.0153
Epoch 44/50
671/671 - 1s - loss: 0.0152


Epoch 18/50
671/671 - 1s - loss: 6.1602e-05
Epoch 19/50
671/671 - 1s - loss: 6.2053e-05
Epoch 20/50
671/671 - 1s - loss: 6.2733e-05
Epoch 21/50
671/671 - 1s - loss: 6.3630e-05
Epoch 22/50
671/671 - 1s - loss: 6.4769e-05
Epoch 23/50
671/671 - 1s - loss: 6.6018e-05
Epoch 24/50
671/671 - 1s - loss: 6.7007e-05
Epoch 25/50
671/671 - 1s - loss: 6.7356e-05
Epoch 26/50
671/671 - 1s - loss: 6.7125e-05
Epoch 27/50
671/671 - 1s - loss: 6.6636e-05
Epoch 28/50
671/671 - 1s - loss: 6.6098e-05
Epoch 29/50
671/671 - 1s - loss: 6.5581e-05
Epoch 30/50
671/671 - 1s - loss: 6.5101e-05
Epoch 31/50
671/671 - 1s - loss: 6.4649e-05
Epoch 32/50
671/671 - 1s - loss: 6.4235e-05
Epoch 33/50
671/671 - 1s - loss: 6.3852e-05
Epoch 34/50
671/671 - 1s - loss: 6.3512e-05
Epoch 35/50
671/671 - 1s - loss: 6.3216e-05
Epoch 36/50
671/671 - 1s - loss: 6.2973e-05
Epoch 37/50
671/671 - 1s - loss: 6.2794e-05
Epoch 38/50
671/671 - 1s - loss: 6.2694e-05
Epoch 39/50
671/671 - 1s - loss: 6.2690e-05
Epoch 40/50
671/671 - 1s - loss:

In [234]:
for market in markets[1:]:
    model_json = multi_step_lstm_models[market].to_json()
    with open('multi_step_lstm_models/multi_step_lstm_model_'+market+'.json', "w") as json_file:
        json_file.write(model_json)
    multi_step_lstm_models[market].save_weights('multi_step_lstm_models/multi_step_lstm_model_'+market+'.h5')

In [255]:
correct_dict_multi_step = {el:None for el in markets[1:]}